In [1]:
%%capture
# Install necessary packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton
!pip install evaluate

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post1: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

<a name="Data"></a>
### Data Prep

In [3]:
squad_prompt = """Below is a context, followed by a question. Write a response that extracts the answer from the context.

### Context:
{}

### Question:
{}

### Answer:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    contexts      = examples["context"]
    questions     = examples["question"]
    answers       = [e["text"][0] for e in examples["answers"]] # examples["answers"]
    texts = []
    for question, context, answer in zip(questions, contexts, answers):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = squad_prompt.format(context, question, answer) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset, concatenate_datasets
dataset = load_dataset("rajpurkar/squad", split="train") #, split=[f"train[{k}%:{k+1}%]" for k in range(0, 100, 100)])
dataset = dataset.select(range(0, len(dataset), 25))

dataset = dataset.map(formatting_prompts_func, batched = True,)

val_dataset = load_dataset("rajpurkar/squad", split="validation")#, split=[f"validation[{k}%:{k+1}%]" for k in range(0, 100, 100)])
val_dataset = val_dataset.select(range(0, len(val_dataset), 100))
val_dataset = val_dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Map:   0%|          | 0/3504 [00:00<?, ? examples/s]

Map:   0%|          | 0/106 [00:00<?, ? examples/s]

In [ ]:
# print(len(dataset), len(val_dataset))

# titles = [ex['title'] for ex in dataset]
# val_titles = [ex['title'] for ex in val_dataset]

# # count each title
# from collections import Counter

# title_counts = Counter(titles)
# val_title_counts = Counter(val_titles)

# # plot
# import matplotlib.pyplot as plt
# plt.bar(title_counts.keys(), title_counts.values(), label='train')
# plt.bar(val_title_counts.keys(), val_title_counts.values(), label='val')
# plt.legend()
# plt.show()

In [4]:
from tqdm.notebook import tqdm

In [5]:
from transformers import AutoTokenizer
from evaluate import load
from unsloth import FastLanguageModel

# Load the SQuAD evaluation metric
squad_metric = load("squad")

# Prepare lists to store predictions and references
predictions_baseline = []
references_baseline = []

# Ensure the model is in inference mode
FastLanguageModel.for_inference(model)

squad_prompt_template = """Below is a context, followed by a question. Write a response that extracts the answer from the context.

### Context:
{}

### Question:
{}

### Answer:
{}"""
# squad_prompt_template = """Below is a question, paired with a context. Write a response that extracts the answer from the context. Only include the extracted text from the context in the answer without any extra words.

# ### Question:
# Which individual worked on projects at Notre Dame that eventually created neoprene?

# ### Context:
# In 1882, Albert Zahm (John Zahm's brother) built an early wind tunnel used to compare lift to drag of aeronautical models. Around 1899, Professor Jerome Green became the first American to send a wireless message. In 1931, Father Julius Nieuwland performed early work on basic reactions that was used to create neoprene. Study of nuclear physics at the university began with the building of a nuclear accelerator in 1936, and continues now partly through a partnership in the Joint Institute for Nuclear Astrophysics.

# ### Answer:
# Father Julius Nieuwland

# ### Question:
# What did Beyoncé announce in January 2010?

# ### Context:
# Beyoncé announced a hiatus from her music career in January 2010, heeding her mother's advice, "to live life, to be inspired by things again". During the break she and her father parted ways as business partners. Beyoncé's musical break lasted nine months and saw her visit multiple European cities, the Great Wall of China, the Egyptian pyramids, Australia, English music festivals and various museums and ballet performances.

# ### Answer:
# a hiatus

# ### Question:
# {}

# ### Context:
# {}

# ### Answer:
# """

# Loop over the first 100 examples of the validation dataset
for i, example in tqdm(enumerate(val_dataset), total=len(val_dataset)):
    # Format the SQuAD prompt using the current example's question and context
    squad_prompt = squad_prompt_template.format(
        example['context'],  # context
        example['question'],   # question
        ""                    # answer - leave blank for generation
    )

    # Tokenize the input
    inputs = tokenizer(
        [squad_prompt],
        return_tensors="pt"
    ).to("cuda")

    # Generate predictions from the model
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True, do_sample=False)

    # Decode the generated tokens into text
    generated_answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    # only get the answer:"
    generated_answer = generated_answer[len(squad_prompt):]

    # Append the generated answer in the required format (id and prediction_text)
    predictions_baseline.append({
        "id": example["id"],  # use the id from the dataset
        "prediction_text": generated_answer.strip()  # the generated text
    })

    # Append the ground truth in the required format (id and answers)
    references_baseline.append({
        "id": example["id"],  # use the same id for the reference
        "answers": {
            "text": example['answers']['text'],  # list of true answers
            "answer_start": example['answers']['answer_start']  # positions of the answers
        }
    })

# Compute the SQuAD metric using predictions and references
results_baseline = squad_metric.compute(predictions=predictions_baseline, references=references_baseline)

# Print the results
print(results_baseline)

  0%|          | 0/106 [00:00<?, ?it/s]

{'exact_match': 16.9811320754717, 'f1': 35.78720510778484}


In [10]:
# combine val_dataset (to_dict) with predictions
data = val_dataset.to_dict()
data["predictions"] = [p["prediction_text"] for p in predictions_baseline]

import pickle

with open("predictions_baseline_worse.pickle", "wb") as f:
    pickle.dump(data, f)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [6]:
FastLanguageModel.for_training(model)
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = False, # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

FastLanguageModel.for_training(model)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = val_dataset,
    # compute_metrics=squad_metric.compute,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        # max_steps = 200,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        # evaluation_strategy = "steps",
    ),
)

Map (num_proc=2):   0%|          | 0/3504 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/106 [00:00<?, ? examples/s]

In [8]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
6.529 GB of memory reserved.


In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,504 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 438
 "-____-"     Number of trainable parameters = 83,886,080
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
5,2.081500
10,1.829000
15,1.699900
20,1.647800
25,1.737500
30,1.637400
35,1.673000
40,1.616900
45,1.490100
50,1.530500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
5,2.081500
10,1.829000
15,1.699900
20,1.647800
25,1.737500
30,1.637400
35,1.673000
40,1.616900
45,1.490100
50,1.530500


In [11]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3664.0236 seconds used for training.
61.07 minutes used for training.
Peak reserved memory = 9.301 GB.
Peak reserved memory for training = 2.772 GB.
Peak reserved memory % of max memory = 63.066 %.
Peak reserved memory for training % of max memory = 18.796 %.


In [ ]:
from transformers import AutoTokenizer
from evaluate import load
from unsloth import FastLanguageModel

# Load the SQuAD evaluation metric
squad_metric = load("squad")

# Prepare lists to store predictions and references
predictions_finetuned = []
references_finetuned = []

# Ensure the model is in inference mode
FastLanguageModel.for_inference(model)

squad_prompt_template = "Context: {}\Question: {}\nAnswer:"

# Loop over the first 100 examples of the validation dataset
for i, example in tqdm(enumerate(val_dataset), total=len(val_dataset)):
    # Format the SQuAD prompt using the current example's context and question
    squad_prompt = squad_prompt_template.format(
        example['context'],  # context
        example['question'],   # question
        ""                    # answer - leave blank for generation
    )

    # print(squad_prompt)

    # Tokenize the input
    inputs = tokenizer(
        [squad_prompt],
        return_tensors="pt"
    ).to("cuda")

    # Generate predictions from the model
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True, do_sample=False)

    # Decode the generated tokens into text
    generated_answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    # only get the answer part:
    generated_answer = generated_answer[len(squad_prompt):]
    # print(generated_answer)

    # Append the generated answer in the required format (id and prediction_text)
    predictions_finetuned.append({
        "id": example["id"],  # use the id from the dataset
        "prediction_text": generated_answer.strip()  # the generated text
    })

    # Append the ground truth in the required format (id and answers)
    references_finetuned.append({
        "id": example["id"],  # use the same id for the reference
        "answers": {
            "text": example['answers']['text'],  # list of true answers
            "answer_start": example['answers']['answer_start']  # positions of the answers
        }
    })

  0%|          | 0/106 [00:00<?, ?it/s]

In [ ]:
# Compute the SQuAD metric using predictions and references
results_finetuned = squad_metric.compute(predictions=predictions_finetuned, references=references_finetuned)

# Print the results
print(results_finetuned)

{'exact_match': 82.0754716981132, 'f1': 92.66042133966661}


In [ ]:
# combine val_dataset (to_dict) with predictions
data = val_dataset.to_dict()
data["predictions"] = [p["prediction_text"] for p in predictions_finetuned]

import pickle

with open("predictions_finetuned.pickle", "wb") as f:
    pickle.dump(data, f)

In [ ]:
# save trainer_stats
import pickle

with open("trainer_stats.pickle", "wb") as f:
    pickle.dump(trainer_stats, f)

In [ ]:
for i in range(10):
  print(100*'-')
  print(predictions[i]['prediction_text'], "\n", references[i]['answers']['text'])

----------------------------------------------------------------------------------------------------
Denver Broncos 
 ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']
----------------------------------------------------------------------------------------------------
Panthers 
 ['Carolina Panthers', 'Carolina Panthers', 'Carolina Panthers']
----------------------------------------------------------------------------------------------------
Santa Clara 
 ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."]
----------------------------------------------------------------------------------------------------
Denver Broncos 
 ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']
----------------------------------------------------------------------------------------------------
gold 
 ['gold', 'gold', 'gold']
----------------------------------------------------------------------------------------------------
Golden 

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# squad_prompt = You MUST copy from above!

inputs = tokenizer(
[
    squad_prompt.format(
        "In which region tribe were large settlements discovered?", # question
        "Terra preta (black earth), which is distributed over large areas in the Amazon forest, is now widely accepted as a product of indigenous soil management. The development of this fertile soil allowed agriculture and silviculture in the previously hostile environment; meaning that large portions of the Amazon rainforest are probably the result of centuries of human management, rather than naturally occurring as has previously been supposed. In the region of the Xingu tribe, remains of some of these large settlements in the middle of the Amazon forest were found in 2003 by Michael Heckenberger and colleagues of the University of Florida. Among those were evidence of roads, bridges and large plazas.", # context
        "", # answer - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is a question, paired with a context. Write a response that extracts the answer from the context.


### Question:
In which region tribe were large settlements discovered?

### Context:
Terra preta (black earth), which is distributed over large areas in the Amazon forest, is now widely accepted as a product of indigenous soil management. The development of this fertile soil allowed agriculture and silviculture in the previously hostile environment; meaning that large portions of the Amazon rainforest are probably the result of centuries of human management, rather than naturally occurring as has previously been supposed. In the region of the Xingu tribe, remains of some of these large settlements in the middle of the Amazon forest were found in 2003 by Michael Heckenberger and colleagues of the University of Florida. Among those were evidence of roads, bridges and large plazas.

### Answer:
the Xingu tribe<|end_of_text|>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**